In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
from mediapipe.python.solutions.holistic import Holistic

2025-07-04 20:20:50.354853: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-04 20:20:52.908632: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-04 20:20:54.083413: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751656854.936869    6809 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751656855.139645    6809 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1751656857.152806    6809 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [2]:
from typing import Tuple
from typing import Any

In [3]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [4]:
def mediapipe_detection(image: 'np.ndarray', model) -> Tuple['np.ndarray', object]:
    # Convert BGR to RGB
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # Disable write access
    rgb_image.flags.writeable = False
    results = model.process(rgb_image)
    rgb_image.flags.writeable = True
    # Convert RGB back to BGR
    output_image = cv2.cvtColor(rgb_image, cv2.COLOR_RGB2BGR)
    return output_image, results

In [5]:
# Initialize drawing and model modules
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic
mp_face_mesh = mp.solutions.face_mesh
mp_hands = mp.solutions.hands

# Define drawing style
DRAWING_STYLES = {
    "face": (
        mp_drawing.DrawingSpec(color=(80, 110, 10), thickness=1, circle_radius=1),
        mp_drawing.DrawingSpec(color=(80, 256, 121), thickness=1, circle_radius=1)
    ),
    "pose": (
        mp_drawing.DrawingSpec(color=(80, 22, 10), thickness=1, circle_radius=1),
        mp_drawing.DrawingSpec(color=(80, 44, 121), thickness=1, circle_radius=1)
    ),
    "left_hand": (
        mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=1, circle_radius=1),
        mp_drawing.DrawingSpec(color=(121, 44, 250), thickness=1, circle_radius=1)
    ),
    "right_hand": (
        mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=1, circle_radius=1),
        mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=1, circle_radius=1)
    )
}


In [6]:
def draw_part(image, landmark, connections, style: tuple):
    """Utility function to draw a landmark with style."""
    mp_drawing.draw_landmarks(
        image,
        landmark,
        connections,
        style[0],
        style[1]
    )

def draw_styled_landmarks(image: 'np.ndarray', results: Any) -> None:
    """
    Draws styled landmarks for face, pose, and both hands using MediaPipe results.

    Args:
        image (np.ndarray): The image/frame to draw on.
        results (object): The result object returned by MediaPipe Holistic.
    """
    if results.face_landmarks:
        draw_part(image, results.face_landmarks, mp_face_mesh.FACEMESH_TESSELATION, DRAWING_STYLES["face"])
    
    if results.pose_landmarks:
        draw_part(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, DRAWING_STYLES["pose"])
    
    if results.left_hand_landmarks:
        draw_part(image, results.left_hand_landmarks, mp_hands.HAND_CONNECTIONS, DRAWING_STYLES["left_hand"])
    
    if results.right_hand_landmarks:
        draw_part(image, results.right_hand_landmarks, mp_hands.HAND_CONNECTIONS, DRAWING_STYLES["right_hand"])


In [7]:
import cv2

for i in range(5):  # Check first 5 camera indexes
    cap = cv2.VideoCapture(i)
    
    if cap.isOpened():
        print(f"Camera found at index {i}")
        cap.release()


[ WARN:0@77.865] global cap_v4l.cpp:913 open VIDEOIO(V4L2:/dev/video0): can't open camera by index
[ERROR:0@77.987] global obsensor_uvc_stream_channel.cpp:158 getStreamChannelGroup Camera index out of range


Camera found at index 1


[ WARN:0@78.209] global cap_v4l.cpp:913 open VIDEOIO(V4L2:/dev/video2): can't open camera by index
[ERROR:0@78.212] global obsensor_uvc_stream_channel.cpp:158 getStreamChannelGroup Camera index out of range
[ WARN:0@78.212] global cap_v4l.cpp:913 open VIDEOIO(V4L2:/dev/video3): can't open camera by index
[ERROR:0@78.216] global obsensor_uvc_stream_channel.cpp:158 getStreamChannelGroup Camera index out of range
[ WARN:0@78.217] global cap_v4l.cpp:913 open VIDEOIO(V4L2:/dev/video4): can't open camera by index
[ERROR:0@78.221] global obsensor_uvc_stream_channel.cpp:158 getStreamChannelGroup Camera index out of range


In [9]:
cap = cv2.VideoCapture(1)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)

        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

I0000 00:00:1751656956.739381    6809 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1751656956.740554    7096 gl_context.cc:369] GL version: 3.0 (OpenGL ES 3.0 Mesa 24.0.9-0ubuntu0.3), renderer: Mesa Intel(R) HD Graphics 3000 (SNB GT2)
W0000 00:00:1751656956.831592    7091 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1751656956.876380    7094 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1751656956.880811    7091 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1751656956.883303    7093 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

# Extracting keypoints

In [11]:

def extract_keypoints(results):
    
    pose = np.array([[lm.x, lm.y, lm.z, lm.visibility] for lm in results.pose_landmarks.landmark]) \
        .flatten() if results.pose_landmarks else np.zeros(33 * 4)

    face = np.array([[lm.x, lm.y, lm.z] for lm in results.face_landmarks.landmark]) \
        .flatten() if results.face_landmarks else np.zeros(468 * 3)

    lh = np.array([[lm.x, lm.y, lm.z] for lm in results.left_hand_landmarks.landmark]) \
        .flatten() if results.left_hand_landmarks else np.zeros(21 * 3)

    rh = np.array([[lm.x, lm.y, lm.z] for lm in results.right_hand_landmarks.landmark]) \
        .flatten() if results.right_hand_landmarks else np.zeros(21 * 3)

    # One-handed sign: Left hand only, mirror and assign to right hand
    if np.all(rh == 0) and not np.all(lh == 0):
        lh_reshaped = lh.reshape(21, 3)
        lh_reshaped[:, 0] = 1 - lh_reshaped[:, 0]  # Mirror X-axis
        rh = lh_reshaped.flatten()
        lh = np.zeros(21 * 3)

    # Two-handed sign: normalize to keep "active" hand on right
    elif not np.all(lh == 0) and not np.all(rh == 0):
        lh_reshaped = lh.reshape(21, 3)
        rh_reshaped = rh.reshape(21, 3)

        # Use Z-axis (depth) range as proxy for hand activity
        lh_z_range = lh_reshaped[:, 2].ptp()  # ptp = max - min
        rh_z_range = rh_reshaped[:, 2].ptp()

        # If left is more active, mirror both and swap
        if lh_z_range > rh_z_range:
            lh_reshaped[:, 0] = 1 - lh_reshaped[:, 0]  # Mirror X
            rh_reshaped[:, 0] = 1 - rh_reshaped[:, 0]
            lh_reshaped, rh_reshaped = rh_reshaped, lh_reshaped  # Swap

        lh = lh_reshaped.flatten()
        rh = rh_reshaped.flatten()

    return np.concatenate([pose, face, lh, rh])  # Final feature vector shape: (1662,)


# Settng up folders for collection

In [14]:

DATA_PATH = "data"
actions = ["hello", "yes", "no", "stop", "thank you"]

no_sequences = 15       
sequence_length = 150   
video_duration = 5     
start_folder = 0


In [15]:
for action in actions:
    action_path = os.path.join(DATA_PATH, action)
    os.makedirs(action_path, exist_ok=True)

    # Determine starting index (default is 30 if folder is empty)
    existing = [int(d) for d in os.listdir(action_path) if d.isdigit()]
    start_idx = max(existing, default=30) + 1

    # Create new sequence folders
    for i in range(start_idx, start_idx + no_sequences):
        os.makedirs(os.path.join(action_path, str(i)), exist_ok=True)


In [16]:
!pwd

/home/izuchukwu/Downloads/Teulogia2.0


In [ ]:
mp_holistic = mp.solutions.holistic
cap = cv2.VideoCapture(1)
cap.set(cv2.CAP_PROP_FPS, 30)

# Helper functions
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = model.process(image)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results


# Data collection
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    for action in actions:
        for sequence in range(start_folder, start_folder + no_sequences):
            keypoints_sequence = []
            total_frames = int(cap.get(cv2.CAP_PROP_FPS) * video_duration)  # e.g., 300 frames at 30 FPS
            frame_interval = max(1, total_frames // sequence_length)  # e.g., 10 frames apart
            frame_count = 0
            collected_frames = 0

            while collected_frames < sequence_length:
                ret, frame = cap.read()
                if not ret:
                    print(f"Error: Unable to read frame for action {action}, sequence {sequence}")
                    break

                frame_count += 1
                if frame_count % frame_interval == 0:  # Sample every nth frame
                    image, results = mediapipe_detection(frame, holistic)
                    draw_styled_landmarks(image, results)

                    # Display status
                    if collected_frames == 0:
                        cv2.putText(image, 'STARTING COLLECTION', (60, 90),
                                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 4, cv2.LINE_AA)
                        cv2.putText(image, f'Collecting frames for {action} Video Number {sequence}', (15, 12),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                        cv2.imshow('OpenCV Feed', image)
                        cv2.waitKey(500)
                    else:
                        cv2.putText(image, f'Collecting frames for {action} Video Number {sequence} ({collected_frames}/{sequence_length})', (15, 12),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                        cv2.imshow('OpenCV Feed', image)

                    keypoints = extract_keypoints(results)
                    keypoints_sequence.append(keypoints)
                    collected_frames += 1

                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break

            # Save the sequence
            if keypoints_sequence:
                sequence_array = np.array(keypoints_sequence)  # Shape: (30, 1662)
                npy_dir = os.path.join(DATA_PATH, action)
                os.makedirs(npy_dir, exist_ok=True)
                npy_path = os.path.join(npy_dir, f"{sequence}.npy")
                np.save(npy_path, sequence_array)
                print(f"Saved sequence: {npy_path}, shape: {sequence_array.shape}")
            
            # Pause between sequences
            cv2.waitKey(1000)  # 1-second pause to reposition

cap.release()
cv2.destroyAllWindows()

In [17]:
!pwd

/home/izuchukwu/Downloads/Teulogia2.0
